In [1]:
import scipy.io
import numpy as np
from GAN_SDR import ReadSignalFromCsv 
# from GAN_SDR_test50_test_3sigma import matFiles, load_allSDR_norm

In [2]:
matFiles25=["pluto1_0.25meters_run2.mat",
    "pluto2_0.25meters_run2.mat",
    "pluto3_0.25meters_run2.mat",
    "AD9082_0.25meters_run2.mat",
    "AD9082_CC2595_amp1_0.25meters_run2.mat",
    "TI_TRF3705_0.25meters.mat",
    "TI_TRF3705_CC2595_amp1_0.25meters.mat",
    "TI_TRF3722_CC2595_amp1_0.25meters.mat",
    "TI_TRF3722_0.25meters.mat",
    "Pluto1RX_9082TX_0.25meters_NoAGC10Gain_3pkthreshold_348samples.mat",
    "Pluto1RX_9082_CC2595_TX_0.25meters_3pkthreshold_582samples.mat"]

matFiles50=["pluto1_0.5meters_run2.mat",
    "pluto2_0.5meters_run2.mat",
    "pluto3_0.5meters_run2.mat",
    "AD9082_0.5meters_run2.mat",
    "AD9082_CC2595_amp1_0.5meters_run2.mat",
    "TI_TRF3705_0.5meters.mat",
    "TI_TRF3705_CC2595_amp1_0.5meters.mat",
    "TI_TRF3722_CC2595_amp1_0.5meters.mat",
    "TI_TRF3722_0.5meters.mat",
    "Pluto1RX_9082TX_0.5meters_NoAGC10Gain_3pkthreshold_403samples.mat",
    "Pluto1RX_9082_CC2595_TX_0.5meters_3pkthreshold_319samples.mat"]

matFiles = {0.25: matFiles25, 0.5: matFiles50}

def load_allSDR_norm(matFiles, distance=0.25):
    datasets_allSDR=None
    targets_allSDR=None
    samples_perSDR=300
    for matfilei in range(len(matFiles)):
        matfile = matFiles[matfilei]
        datasets_load = scipy.io.loadmat("../data/"+str(distance)+"meters/"+matfile)['packet_equalized_allrecord']
        datasets_load = datasets_load[:samples_perSDR]
        targets_load = matfilei*np.ones(datasets_load.shape[0])
        datasets_allSDR = np.concatenate([datasets_allSDR, datasets_load], axis=0) if matfilei!=0 else datasets_load
        targets_allSDR = np.concatenate([targets_allSDR, targets_load], axis=0) if matfilei!=0 else targets_load
    datasets_allSDR_magn = np.max((np.abs(datasets_allSDR.min()), np.abs(datasets_allSDR.max())))
    datasets_allSDR = datasets_allSDR/datasets_allSDR_magn
    return datasets_allSDR, targets_allSDR

In [10]:
def mp_model(x, a, M, P):
    y = np.zeros_like(x, dtype=complex)
    for m in range(M):
        for p in range(1, P + 1):
            y += a[p-1, m] * x * np.abs(x)**(p-1)
    return y

def create_design_matrix(x, M, P):
    N = len(x)
    X = np.zeros((N, M * P), dtype=complex)
    for m in range(M):
        for p in range(1, P + 1):
            X[:, m * P + (p - 1)] = x * np.abs(x)**(p - 1)
    return X

# Function to refine estimates iteratively
def iterative_refinement(x, r, M, P, iterations=50):
    # Create design matrix once
    X = create_design_matrix(x, M, P)
    h_est = np.ones_like(r)  # Initialize channel estimate
    for _ in range(iterations):
        # Normalize received signal using current channel estimate
        y_est = r / h_est
        # Update MP model coefficients
        a_est = np.linalg.lstsq(X, y_est, rcond=None)[0].reshape(P, M)
        # Apply updated MP model to input signal
        # y = mp_model(x, a_est, M, P)
        y = np.matmul(X, a_est.transpose().reshape(-1, 1)).reshape(-1)
        # Update channel estimate
        h_est = r / y
        # Normalize h_est to unit power (optional but helps in some cases)
        h_est /= np.linalg.norm(h_est) / np.sqrt(len(h_est))
    return a_est, h_est

distance = 0.25
data_all, _ = load_allSDR_norm(matFiles[distance], distance=distance)
print("data_all.shape:{}".format(data_all.shape))

# Example parameters
M = 3  # Memory length
P = 5  # Nonlinearity order
iterations = 10000  # Number of iterations for refinement

# Generate example input signals
Nx = 800
np.random.seed(0)
idata=ReadSignalFromCsv('../data/IDataTime.csv')[0][1]
qdata=-ReadSignalFromCsv('../data/QDataTime.csv')[0][1]
preamble_real = idata.squeeze()
preamble_imag = -qdata.squeeze()
x = preamble_real + 1j*preamble_imag
y = data_all[0][0] + 1j*data_all[0][1] #np.random.randn(Nx) + 1j * np.random.randn(Nx)  # Random complex signal
print("x.shape:{}, y.shape:{}".format(x.shape, y.shape))

# True MP model coefficients
a_true = np.random.randn(P, M) + 1j * np.random.randn(P, M)

# Apply MP model to input signal
# y = mp_model(x, a_true, M, P)

# True Rayleigh channel coefficients
h_true = 1/np.sqrt(2) * (np.random.randn(len(x)) + 1j * np.random.randn(len(x)))

# Noise
N0 = 0.1
n = np.sqrt(N0/2) * (np.random.randn(len(x)) + 1j * np.random.randn(len(x)))

# Received signal
r = h_true * y #+ n

# Perform iterative refinement
a_est_final, h_est_final = iterative_refinement(x, r, M, P, iterations)

print(f"True MP Model Coefficients:\n{a_true}")
print(f"Estimated MP Model Coefficients:\n{a_est_final}")
print(f"\nTrue Channel Coefficients (first 5):\n{h_true[:5]}")
print(f"Estimated Channel Coefficients (first 5):\n{h_est_final[:5]}")


data_all.shape:(3300, 2, 800)
x.shape:(800,), y.shape:(800,)
True MP Model Coefficients:
[[ 1.76405235+0.33367433j  0.40015721+1.49407907j  0.97873798-0.20515826j]
 [ 2.2408932 +0.3130677j   1.86755799-0.85409574j -0.97727788-2.55298982j]
 [ 0.95008842+0.6536186j  -0.15135721+0.8644362j  -0.10321885-0.74216502j]
 [ 0.4105985 +2.26975462j  0.14404357-1.45436567j  1.45427351+0.04575852j]
 [ 0.76103773-0.18718385j  0.12167502+1.53277921j  0.44386323+1.46935877j]]
Estimated MP Model Coefficients:
[[0.00263447+0.00709913j 0.00263437+0.00709922j 0.00263434+0.00709924j]
 [0.00263437+0.00709922j 0.00263447+0.00709913j 0.00263447+0.00709913j]
 [0.00263437+0.00709922j 0.00263434+0.00709924j 0.00263437+0.00709922j]
 [0.00263447+0.00709913j 0.00263447+0.00709913j 0.00263437+0.00709922j]
 [0.00263434+0.00709924j 0.00263437+0.00709922j 0.00263447+0.00709913j]]

True Channel Coefficients (first 5):
[ 0.10956438+0.22971751j  0.26740128+0.70506889j -0.62775932+0.02163876j
 -1.40063461-0.04924403j -0.24

In [11]:
X = create_design_matrix(x, M, P)
print(X.shape)
np.linalg.matrix_rank(X)

(800, 15)


3

In [16]:
import numpy as np
from scipy.optimize import minimize

# Likelihood function
def likelihood(params, x, r, M, P):
    a = params[:P * M].reshape((P, M))
    h = params[P * M:]
    y = mp_model(x, a, M, P)
    r_est = h * y
    error = r - r_est
    return np.sum(np.abs(error)**2)

x = preamble_real + 1j*preamble_imag
y = data_all[0][0] + 1j*data_all[0][1] #np.random.randn(Nx) + 1j * np.random.randn(Nx)  # Random complex signal
print("x.shape:{}, y.shape:{}".format(x.shape, y.shape))

# True MP model coefficients
a_true = np.random.randn(P, M)# + 1j * np.random.randn(P, M)
x = preamble_real
y = mp_model(x, a_true, M, P)
h_true = 1/np.sqrt(2) * (np.random.randn(len(x)))#+ 1j * np.random.randn(len(x)))
r = h_true * y


# Initial parameters
a_init = np.random.randn(P, M) + 1j * np.random.randn(P, M)
h_init = np.ones(Nx, dtype=complex)  # Example for the size of h
params_init = np.hstack([a_init.flatten(), h_init])

# Minimize the likelihood function
result = minimize(likelihood, params_init, args=(x, r, M, P), method='L-BFGS-B')
params_est = result.x

# Extract the estimated MP model coefficients and channel coefficients
a_est = params_est[:P * M].reshape((P, M))
h_est = params_est[P * M:]

print(f"Estimated MP Model Coefficients:\n{a_est}")
print(f"Estimated Channel Coefficients (first 5):\n{h_est[:5]}")
print(f"\nTrue MP Model Coefficients:\n{a_true}")
print(f"True Channel Coefficients (first 5):\n{h_true[:5]}")


x.shape:(800,), y.shape:(800,)
Estimated MP Model Coefficients:
[[-0.30628915 -0.74405501  1.1279928 ]
 [ 0.46882959  0.22979515 -0.43385396]
 [ 1.00753998 -2.94571484  0.45496859]
 [ 1.10906136  1.05708926 -0.41919888]
 [-0.9873429  -0.11413203  0.47740569]]
Estimated Channel Coefficients (first 5):
[1.00121201 0.99387936 0.99799843 1.0002549  0.99907225]

True MP Model Coefficients:
[[ 0.78060167  0.93101016  0.23080284]
 [-0.95218681 -1.05693791  0.01010448]
 [ 1.2908111   0.84343517 -1.03141287]
 [-1.38171995 -0.05707179  0.40980275]
 [ 0.72285238 -0.16194162 -0.11279549]]
True Channel Coefficients (first 5):
[-0.96204544  1.19924218 -0.07206495  0.76773814 -0.13651569]
